# import libriaries

In [22]:
import requests 
from lxml import html
from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm
import sys
from sklearn.ensemble import GradientBoostingRegressor

import numpy as np
import pandas as pd
import time
import csv

import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer, LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler 

from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor

# Main functions


In [2]:
def get_all_cities():
  """
  return 
  - list of all cities for site https://sinoptik.com.ru
  - dictionary like {city: region_of_city}
  """
  url = f'https://sinoptik.com.ru/россия'
  r = requests.Session().get(url).content
  tree = html.fromstring(r)
  city_list = [i.lower().replace(' ','-') for i in tree.xpath("//ul[@class = 'capital_list']//text()") if i not in ['\n']]
  region_list = [i.lower().replace(' ','-') for i in tree.xpath("//div[@class = 'current_regions']//text()") if i not in ['\n']]

  city_dict = {}
  for city in city_list:
    city_dict[city] = city

  for region in region_list[:10]:
    url = f"https://sinoptik.com.ru/россия/{region.replace(' ','-').lower()}"
    r = requests.Session().get(url).content
    tree = html.fromstring(r)
    region_city = [i.lower().replace(' ','-') for i in tree.xpath("//section[@class = 'weather__on_big_cities']//text()") if i not in ['\n']][2:]
    city_list += region_city

    for city in region_city:
      city_dict[city] = region

  return city_list, city_dict

In [4]:
def get_date(date, city = 'москва', city_of_region = 'Москва'):
  """
  return update dataframe (df) by addition a news records of city
  """
  global df

  url = f'https://sinoptik.com.ru/погода-{city.lower()}/{date}'
  r = requests.Session().get(url).content
  tree = html.fromstring(r)
  table_info = tree.xpath("//div[@class = 'weather__article_main_right']//text()")

  f = [i.replace('\n','') for i in table_info if i not in ['\n','0',]][4:]
  for i in range(0, len(f), 11)[:-1]:
    time = f[i] + f[i+1]
    describe = f[i+2]
    temperature = f[i+3]
    feels = f[i+4]
    pressure = f[i+5]
    humidity = f[i+6]
    wind_side = f[i+8].split(', ')[0]
    wind = f[i+8].split(', ')[1]
    chance_of_percipt = f[i+11-1]
    forecast = f[-1]



    df = df.append({
        'date_time': date+' '+ time,
        'region': city_of_region,
        'city': city,
        'describe': describe,
        'Temperature, °C': temperature,
        'feels like': feels,
        'Pressure, mm': pressure,
        'Humidity, %' : humidity,
        'wind side': wind_side,
        'Wind, mps': wind,
        'Chance of precipit., %': chance_of_percipt,
        'popular forecast': forecast}, ignore_index=True)
  return df

# Run functions

The next block of code will finish in +- 1 hour. If you dont want wait, you can passed this step. You have a complete file that runs in the following steps

## 1 variant

In [30]:
df = pd.DataFrame(columns = ['date_time', 'region', 'city', 'describe', 'Temperature, °C',
                              'feels like', 'Pressure, mm', 'Humidity, %',
                              'wind side', 'Wind, mps', 'Chance of precipit., %',
                              'popular forecast'])

dates = pd.date_range(start="2021-12-01",end=datetime.today()\
                      .strftime('%Y-%m-%d')).strftime('%Y-%m-%d')

city_list, city_dict = get_all_cities()

with tqdm(total=len(dates)*len(city_list), file=sys.stdout) as pbar:
  for i in range(len(dates)):
      for city in city_list:
        get_date(dates[i], city = city,  city_of_region = city_dict[city])
        pbar.set_description('processed: %d' % (1 + i))
        pbar.update(1)

processed: 12: 100%|██████████| 4812/4812 [56:28<00:00,  1.42it/s]


Run this step by getting full information

## Second variant

In [65]:
!wget 'https://github.com/beaver68/weather-parser/raw/main/weather_2021.zip'
!unzip /content/weather_2021.zip

df = pd.read_csv('weather_2021.csv')
df.shape

--2021-12-12 11:34:17--  https://github.com/beaver68/weather-parser/raw/main/weather_2021.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/beaver68/weather-parser/main/weather_2021.zip [following]
--2021-12-12 11:34:17--  https://raw.githubusercontent.com/beaver68/weather-parser/main/weather_2021.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 645708 (631K) [application/zip]
Saving to: ‘weather_2021.zip’

weather_2021.zip    100%[===================>] 630.57K  --.-KB/s    in 0.006s  

2021-12-12 11:34:17 (96.5 MB/s) - ‘weather_2021.zip’ saved [645708/645708]

Archive:  /content/weather_20

(38304, 12)

In [66]:
df.head()

,date_time,region,city,describe,"Temperature, °C",feels like,"Pressure, mm","Humidity, %",wind side,"Wind, mps","Chance of precipit., %",popular forecast
0,2021-12-01 02:00:00,москва,москва,"Сплошная облачность, снег",-2,-9,723,79,Юго-западный,7.2,0,В первый день зимы почитается память святых Пл...
1,2021-12-01 05:00:00,москва,москва,"Сплошная облачность, сильный снег",-3,-9,725,81,Юго-западный,6.9,0,В первый день зимы почитается память святых Пл...
2,2021-12-01 08:00:00,москва,москва,"Сплошная облачность, снег",-3,-10,726,81,Юго-западный,6.7,0,В первый день зимы почитается память святых Пл...
3,2021-12-01 11:00:00,москва,москва,"Сплошная облачность, снег",-3,-9,727,81,Юго-западный,6.1,0,В первый день зимы почитается память святых Пл...
4,2021-12-01 14:00:00,москва,москва,"Сплошная облачность, снег",-3,-9,728,81,Юго-западный,5.8,0,В первый день зимы почитается память святых Пл...


# Change column type

In [32]:
df['date_time'] = pd.to_datetime(df.date_time)
df.set_index('date_time', inplace=True)

df['Temperature, °C'] = df['Temperature, °C'].apply(lambda x: int(x[:-1]))
df['feels like'] = df['feels like'].apply(lambda x: int(x[:-1]))

df['Pressure, mm'] = df['Pressure, mm'].astype(int)
df['Humidity, %'] = df['Humidity, %'].astype(int)
df['Wind, mps'] = df['Wind, mps'].astype(float)

df['Chance of precipit., %'] = df['Chance of precipit., %'].replace({'-':0})

df.head()

,region,city,describe,"Temperature, °C",feels like,"Pressure, mm","Humidity, %",wind side,"Wind, mps","Chance of precipit., %",popular forecast
date_time,,,,,,,,,,,
2021-12-01 02:00:00,москва,москва,"Сплошная облачность, снег",-2,-9,723,79,Юго-западный,7.2,0,В первый день зимы почитается память святых Пл...
2021-12-01 05:00:00,москва,москва,"Сплошная облачность, сильный снег",-3,-9,725,81,Юго-западный,6.9,0,В первый день зимы почитается память святых Пл...
2021-12-01 08:00:00,москва,москва,"Сплошная облачность, снег",-3,-10,726,81,Юго-западный,6.7,0,В первый день зимы почитается память святых Пл...
2021-12-01 11:00:00,москва,москва,"Сплошная облачность, снег",-3,-9,727,81,Юго-западный,6.1,0,В первый день зимы почитается память святых Пл...
2021-12-01 14:00:00,москва,москва,"Сплошная облачность, снег",-3,-9,728,81,Юго-западный,5.8,0,В первый день зимы почитается память святых Пл...


## Save to file 

In [37]:
df.to_csv('weather_2021.csv')

Drop duplicate records

In [33]:
df.drop_duplicates().shape

(36482, 11)

using ML models for forecasting weather temperature

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression

X = pd.DataFrame({'time': [i.timestamp() for i in df.index],
                  #'feels like': df['feels like'],
                  'Pressure, mm': df['Pressure, mm'],
                  'Humidity, %	': df['Humidity, %']
})
y = df['Temperature, °C']
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.33, random_state=42)


In [44]:
mls = {
        'GradientBoostingRegressor': GradientBoostingRegressor(random_state=0), 
        'linear': LinearRegression(), 'RandomForestRegressor':RandomForestRegressor(max_depth=2, random_state=0),
        'LassoCV': LassoCV(), 'Ridge': RidgeCV(), 'ElasticNetCV': ElasticNetCV(),
       'ExtraTreesRegressor': ExtraTreesRegressor(n_estimators=100, random_state=0)
        }

In [45]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

d = pd.DataFrame()

for model_name, model in mls.items():
  grid = model
  grid.fit(X_train, y_train)

  grad_boost = pd.DataFrame({
      'r2_train': [r2_score(y_train, grid.predict(X_train)).round(6)],
      'r2_test': [r2_score(y_test, grid.predict(X_test)).round(6)],
      'MAE': [mean_absolute_error(y_test, grid.predict(X_test)).round(6)],
      'MSE': [mean_squared_error(y_test, grid.predict(X_test)).round(6)],
      'MAPE': [mean_absolute_percentage_error(y_test, grid.predict(X_test)).round(6)]}, index = [model_name])
  
  d = d.append(grad_boost)
d

,r2_train,r2_test,MAE,MSE,MAPE
GradientBoostingRegressor,0.411434,0.376680,5.138772,45.966749,inf
linear,0.118257,0.100812,6.355580,66.310689,inf
RandomForestRegressor,0.238009,0.219465,5.805328,57.560562,inf
LassoCV,0.015663,0.012812,6.828736,72.800226,inf
Ridge,0.118111,0.101013,6.354833,66.295843,inf
ElasticNetCV,0.015663,0.012812,6.828736,72.800226,inf
ExtraTreesRegressor,0.971209,0.631580,2.630515,27.169178,inf
